In [1]:
import os
os.chdir(os.getcwd() + '/../../')

In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix

from scripts.utils import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PendragonS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PendragonS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
comments = pd.read_csv('data/comments/preprocessed_comments.csv', index_col=0)
comments = comments.dropna()
comments.head()

,toxic,comment_text
0,0,explanation why the edits make under my userna...
1,0,d'aww ! he match this background colour i be s...
2,0,"hey man , i be really not try to edit war . it..."
3,0,`` more i can not make any real suggestion on ...
4,0,"you , sir , be my hero . any chance you rememb..."


# 1. BoW

In [4]:
# vectorization
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(comments['comment_text'])
y = comments['toxic']

# train val, test split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, stratify=y_temp, random_state=0)

In [5]:
# ONLY TOXIC

terms = vectorizer.get_feature_names()

# sum tfidf frequency of each term through documents
means = X[y==1].mean(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append( (term, means[0,col] ))

ranking = pd.DataFrame(data, columns=['term','mean'])
ranking = ranking.sort_values('mean', ascending=False)
ranking.to_csv('data/comments/toxic_tfidf_rank_unigram.csv')
display(ranking.iloc[:10])

,term,mean
64975,fuck,0.800902
150875,suck,0.313587
171694,wikipedia,0.240486
142653,shit,0.238917
94297,like,0.238786
110616,nigger,0.223944
73105,hate,0.172682
117362,page,0.168955
89860,know,0.162286
59940,faggot,0.162286


In [6]:
# ONLY NON-TOXIC

terms = vectorizer.get_feature_names()

# sum tfidf frequency of each term through documents
means = X[y==0].mean(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append( (term, means[0,col] ))

ranking = pd.DataFrame(data, columns=['term','mean'])
ranking = ranking.sort_values('mean', ascending=False)
ranking.to_csv('data/comments/non_toxic_tfidf_rank_unigram.csv')
display(ranking.sort_values('mean', ascending=False).iloc[:10])

,term,mean
19010,article,0.500405
117362,page,0.377732
171694,wikipedia,0.310160
153745,talk,0.263549
165101,use,0.219779
97965,make,0.195755
53932,edit,0.186509
86599,just,0.175690
94297,like,0.173403
138718,say,0.163817
